<a href="https://colab.research.google.com/github/yash2998chhabria/Rnn-Nlp/blob/master/Django_Runnnable_rnn_scrapped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Django-Setting run

In [1]:
import spacy
import torch
import torchtext
import re
import torch.nn as nn

Only For Drive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#/content/drive/MyDrive/STATE DICT OF SCRAPPED RNN DATASET/deprnnscrapped_state_dic

In [5]:
NEW_TEXT =  torch.load("/content/drive/MyDrive/STATE DICT OF SCRAPPED RNN DATASET/vocabvec")

In [18]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])

In [6]:
class RNN(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, 
                 output_dim, n_layers, bidirectional, dropout):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers = n_layers, 
                           bidirectional = bidirectional, dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        
        self.dropout = nn.Dropout(dropout)

        
    def forward(self, text):
        
        embedded = self.dropout(self.embedding(text))
        
        output, hidden = self.rnn(embedded)
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
       
        return self.fc(hidden.squeeze(0))

In [10]:
input_dim = len(NEW_TEXT)

embedding_dim = 100

hidden_dim = 20
output_dim = 1

n_layers = 2
bidirectional = True

dropout = 0.5

In [11]:
new_model = RNN(input_dim, 
            embedding_dim, 
            hidden_dim, 
            output_dim, 
            n_layers, 
            bidirectional, 
            dropout)

In [12]:
new_model.load_state_dict(torch.load('/content/drive/MyDrive/STATE DICT OF SCRAPPED RNN DATASET/deprnnscrapped_state_dic'))

<All keys matched successfully>

In [13]:
new_model.eval()

RNN(
  (embedding): Embedding(25002, 100)
  (rnn): GRU(100, 20, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=40, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

#Django views

In [14]:
sentence = 'The world is a sad place.'

In [19]:
tokenized = [tok.text for tok in nlp.tokenizer(sentence)]

In [20]:
indexed = [NEW_TEXT.stoi[t] for t in tokenized]

In [21]:
tensor = torch.LongTensor(indexed)

In [22]:
tensor = tensor.unsqueeze(1)

In [23]:
prediction = torch.sigmoid(new_model(tensor))

In [24]:
if prediction.item() > 0.5:
  print("Depression Symptoms Detected")
else:
  print("No Symptoms Detected")  

prediction.item()

Depression Symptoms Detected


0.6122962236404419